In [32]:
import json
import re
import numpy as np
from collections import Counter
with open("recipes_cleaned_title.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data))
print(data[0].keys())


4090
dict_keys(['_id', 'name', 'description', 'imageURL', 'cuisine', 'preTimeMinutes', 'cookTimeMinutes', 'totalTimeMinutes', 'servings', 'ingredients', 'instructions', 'nutritionInfo', 'tags', 'createdAt', 'updatedAt'])


In [33]:
for recipe in data:
    recipe["name"] = recipe["name"].strip()
    for ing in recipe["ingredients"]:
        ing["name"] = ing["name"].lower().strip()


In [34]:
name_lengths = [len(recipe["name"].split()) for recipe in data]
max_name = max(data, key=lambda r: len(r["name"].split()))
all_words = [word for r in data for word in r["name"].split()]


In [35]:
ingredient_counts = [len(recipe["ingredients"]) for recipe in data]


ingredient_counter = Counter()
for r in data:
    for ing in r["ingredients"]:
        ingredient_counter[ing["name"]] += 1
ingredient_counter.most_common(10)

missing_counts = {}
for recipe in data:
    for key, value in recipe.items():
        if value in [None, "", [], {}]:
            missing_counts[key] = missing_counts.get(key, 0) + 1

all_steps = []
for r in data:
    if "steps" in r and isinstance(r["steps"], list):
        all_steps.extend(r["steps"])

words = [w for step in all_steps for w in re.findall(r'\w+', step.lower())]
step_word_counts = Counter(words).most_common(20)

In [36]:
mean_len = np.mean(name_lengths)
median_len = np.median(name_lengths)
std_len = np.std(name_lengths)

# he so = 2
k = 2
# cong thuc tinh outlier > mean + he_so. do_lech_chuan hoac < mean - he_so. do_lech_chuan
outlier_titles = [
    r for r in data
    if len(r["name"].split()) < mean_len - k * std_len or len(r["name"].split()) > mean_len + 2 * std_len
]

mean_ing = np.mean(ingredient_counts)
std_ing = np.std(ingredient_counts)

outlier_ingredients = [
    r for r in data
    if len(r["ingredients"]) > mean_ing + k * std_ing or len(r["ingredients"]) < mean_ing - 2 * std_ing
]


outlier_titles_info = [
    {"ten": r["name"], "so_tu": len(r["name"].split())}
    for r in outlier_titles
]


outlier_ingredients_info = [
    {"ten": r["name"], "so_nguyen_lieu": len(r["ingredients"])}
    for r in outlier_ingredients
]


In [37]:
result = {
    "tong_so_mon_an": len(data),

    # Tên món dài nhất theo số chữ
    "ten_mon_dai_nhat": max_name.get("name"),

    # Thống kê số chữ trong tên món
    "trung_binh_so_chu_trong_ten": round(mean_len, 2),
    "trung_vi_so_chu_trong_ten": median_len,
    "do_lech_chuan_so_chu_trong_ten": round(std_len, 2),

    # Thống kê nguyên liệu
    "so_nguyen_lieu_trung_binh": round(mean_ing, 2),
    "do_lech_chuan_nguyen_lieu": round(std_ing, 2),

    # Top 10 nguyên liệu phổ biến
    "top_nguyen_lieu_pho_bien": ingredient_counter.most_common(10),

    # Thống kê số trường bị trống
    "so_truong_bi_trong": dict(missing_counts),
    "ten_qua_dai": outlier_titles_info,
    "so luong ten qua dai":  len(outlier_titles_info),
    "mon_an_qua_nhieu_nguyen_lieu": outlier_ingredients_info,
    "so luong mon qua nhieu nguyen lieu": len(outlier_ingredients_info),
}

# In ra kết quả
import pprint
pprint.pprint(result, sort_dicts=False)


{'tong_so_mon_an': 4090,
 'ten_mon_dai_nhat': 'Salad Trái Cây: Chuối + Dưa Leo + Xà Lách + Phô Mai Con '
                     'Bò Cười Vị Dâu + 1 Ít Sữa Đặc',
 'trung_binh_so_chu_trong_ten': np.float64(5.74),
 'trung_vi_so_chu_trong_ten': np.float64(5.0),
 'do_lech_chuan_so_chu_trong_ten': np.float64(2.45),
 'so_nguyen_lieu_trung_binh': np.float64(6.63),
 'do_lech_chuan_nguyen_lieu': np.float64(3.65),
 'top_nguyen_lieu_pho_bien': [('đường', 590),
                              ('gia vị', 473),
                              ('hành lá', 416),
                              ('muối', 382),
                              ('dầu ăn', 299),
                              ('tỏi', 295),
                              ('cà rốt', 290),
                              ('hành tây', 232),
                              ('nước mắm', 229),
                              ('cà chua', 210)],
 'so_truong_bi_trong': {'preTimeMinutes': 4089,
                        'tags': 4089,
                        'cookTimeMinut

In [38]:
from collections import Counter
import re

# Tách tất cả các từ trong tên món
all_words = []
for recipe in data:
    words = re.findall(r'\b\w+\b', recipe["name"].lower())
    all_words.extend(words)

word_freq = Counter(all_words)
common_words = word_freq.most_common(100)  # top 100 từ phổ biến
pprint.pprint(common_words, sort_dicts=False)

[('thịt', 498),
 ('gà', 484),
 ('chiên', 481),
 ('cơm', 418),
 ('tôm', 367),
 ('mì', 307),
 ('cá', 305),
 ('canh', 291),
 ('cháo', 290),
 ('xào', 287),
 ('rau', 276),
 ('đậu', 275),
 ('trộn', 251),
 ('nước', 250),
 ('trứng', 245),
 ('chè', 237),
 ('nấu', 235),
 ('bánh', 214),
 ('chua', 209),
 ('nấm', 202),
 ('bò', 186),
 ('sốt', 180),
 ('sữa', 180),
 ('cà', 179),
 ('heo', 178),
 ('cải', 175),
 ('và', 160),
 ('rang', 155),
 ('hạt', 146),
 ('xôi', 144),
 ('lá', 143),
 ('mắm', 136),
 ('kho', 129),
 ('luộc', 129),
 ('xanh', 129),
 ('chay', 127),
 ('củ', 124),
 ('nướng', 120),
 ('nem', 120),
 ('hầm', 118),
 ('giòn', 116),
 ('khoai', 115),
 ('chả', 113),
 ('trà', 111),
 ('đỏ', 108),
 ('tây', 97),
 ('bắp', 94),
 ('ăn', 88),
 ('bí', 88),
 ('biển', 86),
 ('tươi', 83),
 ('bún', 82),
 ('ngọt', 81),
 ('rong', 77),
 ('bình', 77),
 ('với', 73),
 ('tiêu', 73),
 ('xay', 71),
 ('mực', 69),
 ('trái', 69),
 ('chuối', 68),
 ('hoa', 68),
 ('gỏi', 68),
 ('bơ', 68),
 ('tỏi', 68),
 ('dừa', 66),
 ('ớt', 66),
 

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

titles = [r["name"].lower() for r in data]

vectorizer = CountVectorizer(ngram_range=(2, 4), max_features=200)

X = vectorizer.fit_transform(titles)

ngrams_freq = zip(vectorizer.get_feature_names_out(), X.sum(axis=0).tolist()[0])

sorted_ngrams = sorted(ngrams_freq, key=lambda x: x[1], reverse=True)

for ngram, freq in sorted_ngrams:
    print(f"{ngram}: {freq}")

chiên giòn: 95
gà chiên: 90
cơm chiên: 89
rong biển: 65
rau củ: 63
thịt heo: 63
cà chua: 61
cà rốt: 61
thịt bò: 59
mì lá: 57
nấu tôm: 57
cháo thịt: 56
tôm thịt: 56
nước mắm: 54
phô mai: 52
đùi gà: 52
thập cẩm: 51
trái cây: 51
ức gà: 49
lá tươi: 48
mì lá tươi: 48
bò xào: 47
khoai tây: 47
trứng chiên: 44
yến mạch: 43
kho tiêu: 42
canh chua: 40
hạt sen: 40
chiên nước: 39
cánh gà: 39
sữa chua: 39
đơn giản: 38
chiên nước mắm: 37
cơm nhà: 37
khoai lang: 37
rau câu: 37
chả cá: 36
đậu xanh: 36
cá hồi: 35
nước tương: 35
đậu hũ: 35
bí đỏ: 34
bánh mì: 33
bánh tráng: 33
nước ép: 33
trà sữa: 33
canh rau: 32
thịt rang: 32
ớt chuông: 32
cánh gà chiên: 31
thịt gà: 31
thịt vai: 31
bắp cải: 30
hàn quốc: 30
xào tôm: 30
chè dưỡng: 29
chè dưỡng nhan: 29
dưỡng nhan: 29
gạo lứt: 29
trứng gà: 29
chè trôi: 28
gà chiên nước: 28
gà chiên nước mắm: 28
rau muống: 28
chiên trứng: 27
chua ngọt: 27
chè bưởi: 27
hạt điều: 27
sốt cà: 27
xúc xích: 27
đá xay: 27
chè trôi nước: 26
cơm bento: 26
sinh tố: 26
sữa hạt: 26
trô

In [43]:
import random
black_list = [
    "cách làm", "của annie", "của annie vo", "của pham", "của pham huyen", "của bếp", "của rose", "của rose truong",
    "bòn bon", "của bòn bon", "của minh", "của bảo", "của huyen", "của dory", "của huyen le", "của huyen le tran", "của bảo bình", "của bếp nhà", "của quân nguyễn", "của bếp nhà cọp", "của nhà vân", "của sơn panda", "của hoàng", "làm món sp", "của phan", "của phan bao van", "của minh hayes", "của kim", "của huỳnh", "của huỳnh phát", "của kim dung", "của bếp của", "của quỳnh", "của trần", "của hoàng thị", "của bếp của quỳnh", "của hoàng thị tố", "hoàng thị tố hà", "của bếp hien dang", "thị tố hà"
    , "thu mèo", "của hân hân", "cho bé", "món", "nhà cọp", "eat clean", "của", "le tran", "huyen", "vo", "truong", "hayes", "le tran", "bao van", "dễ làm", "?", "!", "thơm phức", "ăn dặm", ":"

]

def clean_title(title, blacklist):
    title = title.lower()

    # Bắt tất cả cụm 'của <tên>'
    title = re.sub(r"của [a-zà-ỹ\s]{2,30}", "", title)

    for phrase in blacklist:
        title = re.sub(r"\b" + re.escape(phrase) + r"\b", "", title)

    title = re.sub(r"\s+", " ", title).strip()
    return title.title()



for r in data:
    r["name"] = clean_title(r["name"], black_list)
with open("recipes_cleaned_title.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

    # Kiểm tra lại 5 dòng đầu sau khi clean


for r in random.sample(data, 50):
    print(r["name"])




Cháo Hạt Vỡ Thịt Gà 🍗 Nấm Đùi Gà
Mực Mai Xào Ớt Chuông .
🌱 Nước Mát Mủ Trôm Phổ Tai Kỷ Tử
Chè Khúc Bạch Hoa Quả 🍒🥣
Cá Khoai Nấu Ngót
Salad Đậu Gà Đậu Đỏ Rong Nho
Mắm Tép Rang Tôm Khô, Thịt
Cá Mắm Sốt Cà Chua (Chua Ngọt) 19T (Luong Thu Hoai)
Trà Táo Xanh Bạc Hà
Cơm Trứng Mây Chả Gà Xúc Xích
Cơm Nắm Sốt Choco
Cháo Thịt Bằm Cải Ngọt
Bánh Nhân Chả Giò Chiên Giòn
Ngồng Su Hào Xào Thịt Bò
Ức Gà Sốt Cam
Mực Sữa Rim Nước Mắm
Gà Chiên Bằng Nckd
Gà Áp Chảo Sốt Chanh Leo, Kèm Salad
Rau Tiến Vua Xào Heo Rừng
Cháo Yến Mạch Bí Ngô (Ngày 8)
Nước Ép Củ Dền Và Dưa Hấu Bổ Máu Đẹp Da
Custard (Kem Hột Gà Trứng Sữa)
Sữa Đậu Xanh Mix Kỷ Tử, Táo Đỏ
Cá Trê Kho Gừng .
Xôi Cốm Hạt Sen
Bầu Um Hột Vịt Lộn
Bún Chả Nem Nướng - Nhanh Gọn
Rau Mầm Đá Luộc
Nước Cam Quýt Vắt
Bánh Mì Xá Xíu
Thịt Xay Rang Tóp Mỡ
Mì Ăn Liền Vị Kim Chi Nấu Măng Chua
Củ Sen Chiên Giòn Lắc Phô Mai
Sinh Tố Đào Nguyên
Chè Thập Cẩm Hương
Tôm Cháy Tỏi Gambas Al Ajillo
Cá Hồi Chay (Đậu Sốt Tương Ớt Gochujang)
Mực Trứng Luộc Lá Lốt (Canh Mực Lá Lốt